In [147]:
# Web scraping with Selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException

import time
from tqdm import tqdm

# Data handling
import pandas as pd
import numpy as np

import os


# Func.

In [2]:
def set_driver():
    options = webdriver.ChromeOptions()
    # options.add_argument('--headless')
    # options.add_argument('--no-sandbox')
    # options.add_argument('--disable-dev-shm-usage')
    driver = webdriver.Chrome(options=options)
    return driver

In [3]:
def get_element_value(driver, xpath):
    element = driver.find_element(By.XPATH, xpath)
    return element.text

In [4]:
def set_columns(driver):
    manufacturer_xpath = f"/html/body/div/div[2]/div[3]/div/div/div[2]/div/div[1]/ul/li[{1}]/dl/dd/ul/li[1]/div[1]"
    ingredient_xpath = f"/html/body/div/div[2]/div[3]/div/div/div[2]/div/div[1]/ul/li[{1}]/dl/dd/ul/li[2]/div[1]"
    specification_xpath = f"/html/body/div/div[2]/div[3]/div/div/div[2]/div/div[1]/ul/li[{1}]/dl/dd/ul/li[3]/div[1]"
    description_xpath = f"/html/body/div/div[2]/div[3]/div/div/div[2]/div/div[1]/ul/li[{1}]/dl/dd/ul/li[4]/div[1]"
    
    manufacturer = driver.find_element(By.XPATH, manufacturer_xpath).text
    ingredient = driver.find_element(By.XPATH, ingredient_xpath).text
    specification = driver.find_element(By.XPATH, specification_xpath).text
    description = driver.find_element(By.XPATH, description_xpath).text
    
    return manufacturer, ingredient, specification, description

In [5]:
def get_product_details(driver, index):
    name_btn = driver.find_element(By.XPATH, f"/html/body/div/div[2]/div[3]/div/div/div[2]/div/div[1]/ul/li[{index}]/dl/dt/div[1]/div")
    name_btn.click()

    # Define the XPaths for all required fields
    xpaths = {
        'name': f'/html/body/div/div[2]/div[3]/div/div/div[2]/dl/dt',
        'type': f'/html/body/div/div[2]/div[3]/div/div/div[2]/dl/dd[1]/div[2]/ul/li[1]/span',
        'ingredients': f'/html/body/div/div[2]/div[3]/div/div/div[2]/dl/dd[1]/div[2]/ul/li[2]/span',
        'alcohol_content': f'/html/body/div/div[2]/div[3]/div/div/div[2]/dl/dd[1]/div[2]/ul/li[3]/span',
        'volume': f'/html/body/div/div[2]/div[3]/div/div/div[2]/dl/dd[1]/div[2]/ul/li[4]/span',
        'awards': f'/html/body/div/div[2]/div[3]/div/div/div[2]/dl/dd[1]/div[2]/ul/li[5]/span',
        'additional_info': f'/html/body/div/div[2]/div[3]/div/div/div[2]/dl/dd[1]/div[2]/ul/li[6]/span',
        'product_introduction': '/html/body/div/div[2]/div[3]/div/div/div[2]/dl/dd[2]/div[2]',
        'pairing_food': '/html/body/div/div[2]/div[3]/div/div/div[2]/dl/dd[3]/div[2]',
        'brewery': '/html/body/div/div[2]/div[3]/div/div/div[2]/dl/dd[4]/div[2]/ul/li[1]/span',
        'address': '/html/body/div/div[2]/div[3]/div/div/div[2]/dl/dd[4]/div[2]/ul/li[2]/span',
        'website': '/html/body/div/div[2]/div[3]/div/div/div[2]/dl/dd[4]/div[2]/ul/li[3]/span',
        'contact_info': '/html/body/div/div[2]/div[3]/div/div/div[2]/dl/dd[4]/div[2]/ul/li[4]/span'
    }

    details = {}
    
    # Retrieve each detail by its XPath and handle exceptions if elements are not found
    for key, xpath in xpaths.items():
        try:
            details[key] = driver.find_element(By.XPATH, xpath).text
        except NoSuchElementException:
            details[key] = None  # Set as None if the element is not found

    try:
        return_btn = driver.find_element(By.CLASS_NAME, 'btn-primary')
        return_btn.click()
    except NoSuchElementException:
        print("Return button not found, could not navigate back.")

    return details

In [6]:
def scrape_page(driver):
    # 현재 페이지 내 10개 인덱스 데이터 수집
    data_list = []
    for index in range(1, 11):  # 1~10 인덱스
        try:
            details = get_product_details(driver, index)
            data_list.append(details)
        except NoSuchElementException:
            # 인덱스에 해당하는 요소가 없으면 반복 중단
            break
    return pd.DataFrame(data_list)  # 현재 페이지의 데이터를 DataFrame으로 반환


# Crawling

## 더 술 닷컴: 우리술 찾기

https://thesool.com/front/find/M000000082/list.do

In [77]:
driver = set_driver()

In [8]:
driver.get(f'https://thesool.com/front/find/M000000082/list.do')

In [9]:
df_products = pd.DataFrame()

In [10]:
# 페이지 수집 및 네비게이션
page = 3
while True:
    # time.sleep(1)  # 데이터 로딩 대기
    new_data_df = scrape_page(driver)  # 현재 페이지에서 데이터 수집
    df_products = pd.concat([df_products, new_data_df], ignore_index=True)
    
    # 다음 페이지로 이동
    try:
        next_button = driver.find_element(By.XPATH, f'/html/body/div/div[2]/div[3]/div/div/div[2]/div/div[2]/ul/li[{page}]/a')
        next_button.click()
        page = 3 if page == 7 else page + 1  # 3부터 7까지 순환
    except NoSuchElementException:
        # 다음 페이지 버튼이 없을 경우, 반복 종료
        break

In [11]:
df_products

,name,type,ingredients,alcohol_content,volume,awards,additional_info,product_introduction,pairing_food,brewery,address,website,contact_info
0,블링블링 노을별,리큐르/기타주류,"정제수, 포도과육(국내산100%)증류원액, 정백당, 효소처리스테비아, 포도향(천연향료)",15%,375ml,,,"블링블링 노을별은 영덕 바다의 은하수를 담아낸 우주 술로, 포도의 향과 맛이 조화롭...","파스타,피자,샐러드..",영덕주조,경북 영덕군 강구면 소월1길 16-10,https://smartstore.naver.com/yeongdeok,None
1,너브내 화이트(dry) 와인,과실주,"국산 포 도효모, 설탕, 아황산나트륨, 수트랄로스, 자일리톨",12%,750ml,,,향긋한 포도의 향기가 나는 건강한 포도와 강원도 홍천의 맑은 공기와 깨끗한 환경이 ...,"한식.생선회,초밥..",샤또나드리,강원도 홍천군 서면 팔봉산로 811-28 . 샤또나드리 / 너브내와인 (우 : 25...,https://smartstore.naver.com/nwine,None
2,너디펀치,탁주,"정제수, 찹쌀, 효모, 정제효소",5%,700ml,,,"참쌀의 부드러운 단맛과 최적의 발효온도로 구현한 청량한 산미,저온 숙성을 통한 풍부...",과일.,상주주조,경상북도 상주시 남성로 73-15 1층 (우 : 37196),https://smartstore.naver.com/nerdbrew,None
3,너디로제,탁주,"너디로제:정제수, 찹쌀, 체리, 효모, 정제효소",5.5%,700ml,,,"체리의 풍부함과,애플민트의 시원한 향이 만드는 환상의 하모니의 새콤달콤한 막걸리입니다.","체리빵,각종 디저트류,파스타.",상주주조,경상북도 상주시 남성로 73-15 1층 (우 : 37196),https://smartstore.naver.com/nerdbrew,None
4,가무치 43%,증류주,"정제수, 쌀(국내산), 입국, 효모",43%,375ml,,,감압식 소주와 비교하여 상압식 소주는 향이 다채롭고 화려하다는 특징이 있습니다.하지...,,다농바이오,충청북도 충주,https://smartstore.naver.com/danongbio,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1023,오미자 생막걸리,탁주,"정제수, 백미(국내산), 입국(쌀, 밀), 올리고당, 물엿, 허브(유기농), 오미자...",6.5%,750ml,,2015 찾아가는 양조장,다섯 가지 맛이 난다고 하여 붙여진 이름처럼 문경주조의 오미자 막걸리는 마실 때마다...,"산뜻한 오미자 특유의 산미 향이 코를 자극시키며, 오미자의 특징인 오미의 조화가 잘...",문경주조,경상북도 문경시 동로면 노은리 192번지,https://mgomijasul.modoo.at/,None
1024,오산막걸리,탁주,"쌀(국내산), 국, 효모, 정제수",6%,500ml,,,오산막걸리는 오산에서 재배한 세마쌀을 주원료로 누룩과 물 이외에는 인공감미료를 넣지...,술을 마시고 난 후에도 개운하고 아주 약하게 남는 곡물의 느낌이 음식과의 조화를 잘...,농업회사법인 오산양조㈜,경기도 오산시 시장길 63,http://osansool.com/,None
1025,행운 생유자 동동주,탁주,"쌀(국내산), 소맥분(미국산), 전분당, 정제수, 유자과즙, 치자, 정제효소, 종국...",6%,750ml / 1700ml,,,유자 동동주는 거제도의 특산품 유자를 활용하여 생산한 막걸리다. 유자 추출액을 발효...,술을 마시고 난 후에도 매우 개운하다. 산미와 신선함이 자칫 술 맛을 가볍게 만들 ...,성포양조장,경남 거제시 사등면 지석3길3,http://성포양조장.kr,None
1026,연천 율무 동동주,탁주,"쌀(국내산), 누룩, 효모, 연천 율무, 정제수",14%,750ml,,,경기도 최북단 DMZ의 민통선에 위치한 연천양조장에서 연천 고래실논에서 재배되는 연...,알콜올 도수가 14%로 일반 막걸리의 2배가 넘는 만큼 한여름에는 얼음을 넣어 온더...,농업회사법인 연천양조㈜,경기도 연천군 마산면 청정로 1738번길 15,https://ksool88.modoo.at/?NaPm=ct%3Dkj7xdaia%7...,None


In [179]:
df_products.to_csv(os.getcwd() + '/alcohol.csv', index=False)

In [ ]:
driver.quit()

# Naver

In [188]:
df_products = pd.read_csv(os.getcwd() + '/alcohol.csv')

In [79]:
driver = set_driver()

In [189]:
import pandas as pd
from soynlp.normalizer import *
from difflib import SequenceMatcher
import re

def normalize_text(text):
    # 숫자와 한글, 영문자를 제외한 모든 문자 제거
    cleaned_text = re.sub(r'[^0-9a-zA-Z가-힣]', ' ', text)
    # 공백 정리
    cleaned_text = re.sub(r'\s+', ' ', cleaned_text).strip()
    
    # 반복되는 문자 정규화
    normalized_text = repeat_normalize(cleaned_text, num_repeats=2)
    return normalized_text

def similarity_score(str1, str2):
    str1_normalized = normalize_text(str1)
    str2_normalized = normalize_text(str2)
    return SequenceMatcher(None, str1_normalized, str2_normalized).ratio()

In [190]:
def find_best_match_product(driver, name, volume):
    base_url = "https://search.shopping.naver.com/search/all?query="

    # First search with name and volume
    driver.get(f"{base_url}{name} {volume}")
    best_product_details = None
    highest_similarity = 0

    # Function to extract product details and calculate similarity
    def process_search():
        nonlocal highest_similarity
        nonlocal best_product_details
        for index in range(1, 4):  # Inspect the top 5 products
            try:
                if driver.find_element(By.XPATH, f"/html/body/div/div/div[2]/div/div[3]/div[1]/div[2]/div/div[{index}]/div/div/div[2]/div[3]/span[2]").text == '주류':
                    product_info = WebDriverWait(driver, 3 if index == 1 else 0.5).until(
                        EC.visibility_of_element_located((By.XPATH, f"/html/body/div/div/div[2]/div/div[3]/div[1]/div[2]/div/div[{index}]/div/div/div[2]/div[1]/a"))
                    )
                    title = product_info.get_attribute('title')
                    current_similarity = similarity_score(name, title)
                else:
                    current_similarity = 0
                    
                if current_similarity > highest_similarity:
                    highest_similarity = current_similarity
                    # Extract details only if the current product is more similar than previous ones
                    shipping_info = driver.find_element(By.XPATH, f"/html/body/div/div/div[2]/div/div[3]/div[1]/div[2]/div/div[{index}]/div/div/div[2]/div[2]/strong/span[2]").text.split('\n')[1]
                    price_info = driver.find_element(By.XPATH, f"/html/body/div/div/div[2]/div/div[3]/div[1]/div[2]/div/div[{index}]/div/div/div[2]/div[2]/strong/span[1]").text
                    product_link = product_info.get_attribute('href')
                    best_product_details = (title, price_info, shipping_info, product_link)
            except (TimeoutException, NoSuchElementException):
                driver.refresh()
                continue  # Skip to the next product if any error occurs

    # Process first search
    process_search()

    # Second search without volume to compare results
    driver.get(f"{base_url}{name}")
    process_search()

    # Return details of the most similar product found across both searches
    if highest_similarity < 0.2:  # Check if any sufficiently similar product was found
        return None, None, None, None

    return best_product_details

In [191]:
for i in tqdm(range(len(df_products)), desc="Processing products"):
    product_name, product_price, product_shipping_cost, product_link = find_best_match_product(
        driver, df_products.at[i, 'name'], df_products.at[i, 'volume']
    )
    df_products.at[i, 'Naver Name'] = product_name
    df_products.at[i, 'Naver Price'] = product_price
    df_products.at[i, 'Naver Shipping Cost'] = product_shipping_cost
    df_products.at[i, 'Naver Link'] = product_link

Processing products: 100%|██████████| 1028/1028 [45:48<00:00,  2.67s/it] 


In [192]:
df_products.to_csv(os.getcwd() + '/df_products.csv', index=False)

## Naver: Shopping Using API

차추에 제품 구매 링크 및 최저가 표기: streamlit

In [29]:
import urllib.request
import urllib.parse
import json

In [60]:
with open(os.getcwd() + '/naver_shopping_api.json', 'r') as file:
    data = json.load(file)  # 파일 내용을 JSON으로 읽어서 파싱

client_id = data['client_id']
client_secret = data['client_secret']

In [72]:
def search_product(client_id, client_secret, product_name):
    # URL 인코딩
    encText = urllib.parse.quote(product_name)
    url = f'https://openapi.naver.com/v1/search/shop.json?query={encText}&sort=sim'

    # 요청 객체 생성 및 헤더 설정
    request = urllib.request.Request(url)
    request.add_header("X-Naver-Client-Id", client_id)
    request.add_header("X-Naver-Client-Secret", client_secret)
    
    # 서버 응답
    try:
        response = urllib.request.urlopen(request)
        response_body = response.read()
        data = json.loads(response_body.decode('utf-8'))
        return data
    except urllib.error.HTTPError as e:
        print(f"HTTP Error: {e.code} {e.reason}")
        return None

In [62]:
for product in df_products['name']:
    result = search_product(client_id, client_secret, product)
    if result and result['items']:
        items = result['items']
        if items:
            lowest_price_item = min(items, key=lambda x: int(x['lprice']))
            print(f"Product: {product}, Lowest Price: {int(lowest_price_item['lprice'])}, Link: {lowest_price_item['link']}")
        else:
            print(f"No results found for {product}")
    else:
        print(f"Error searching for {product}")

Product: 블링블링 노을별, Lowest Price: 22000, Link: https://search.shopping.naver.com/gate.nhn?id=47139267802
Product: 너브내 화이트(dry) 와인, Lowest Price: 30000, Link: https://search.shopping.naver.com/gate.nhn?id=82173577812
Product: 너디펀치, Lowest Price: 8190, Link: https://search.shopping.naver.com/gate.nhn?id=44209546332
Product: 너디로제, Lowest Price: 9000, Link: https://search.shopping.naver.com/gate.nhn?id=46649927387
Product: 가무치 43%, Lowest Price: 33440, Link: https://search.shopping.naver.com/gate.nhn?id=47139103151
Product: 호땅, Lowest Price: 2730, Link: https://search.shopping.naver.com/gate.nhn?id=47139132675


KeyboardInterrupt: 

# Pre-Processing

continue..

In [14]:
# 규격/도수 컬럼을 공백으로 분리하여 새로운 데이터프레임 생성
split_df = df_products['규격/도수'].str.split(' / ', expand=True)

# 새로운 규격 및 도수 컬럼 생성
df_products['규격'] = None
df_products['도수'] = None

# 분리된 데이터에서 각 행을 반복하며 규격과 도수 분류
for index, row in split_df.iterrows():
    for item in row:
        if item is not None:
            if 'ml' in item:
                df_products.at[index, '규격'] = item  # 'ml'을 포함하는 경우 규격으로 설정
            elif '%' in item:
                df_products.at[index, '도수'] = item  # '%'를 포함하는 경우 도수로 설정

# 결과 확인
print(df_products)


                  제품명           제조사  \
0            블링블링 노을별          영덕주조   
1     너브내 화이트(dry) 와인         샤또나드리   
2                너디펀치          상주주조   
3                너디로제          상주주조   
4             가무치 43%         다농바이오   
...               ...           ...   
1023         오미자 생막걸리          문경주조   
1024            오산막걸리  농업회사법인 오산양조㈜   
1025       행운 생유자 동동주         성포양조장   
1026        연천 율무 동동주  농업회사법인 연천양조㈜   
1027       이동 생 쌀 막걸리          이동주조   

                                                    주원료  \
0      정제수, 포도과육(국내산100%)증류원액, 정백당, 효소처리스테비아, 포도향(천연향료)   
1                     국산 포 도효모, 설탕, 아황산나트륨, 수트랄로스, 자일리톨   
2                                     정제수, 찹쌀, 효모, 정제효소   
3                            너디로제:정제수, 찹쌀, 체리, 효모, 정제효소   
4                                   정제수, 쌀(국내산), 입국, 효모   
...                                                 ...   
1023  정제수, 백미(국내산), 입국(쌀, 밀), 올리고당, 물엿, 허브(유기농), 오미자...   
1024                                 쌀(국내산), 국, 효모, 정제수   
